In [67]:
from qutip import *
from numpy import pi
import numpy as np
import matplotlib.pyplot as plt

In [68]:
def oracle(x,dim):
    array = [ ]
    if x>dim:
        return("dim error: dim should greater or equal to x")
    for i in range(dim):
        if i+1==x:
            array.append([1])
        else:
            array.append([0])
    return(array)

def diffusor_construct(s):
    #the input state should be ket
    sm = [ ]
    dim = s.shape[0]
    for i in range(dim):
        numi = s[i][0][0]
        sm.append([ ])
        for j in range(dim):
            numj = s[j][0][0]
            sm[i].append(2*np.conj(numi)*numj)
            if i == j:
                sm[i][j] = sm[i][j]-1
    return(sm)

In [69]:
w = 1 #Hamming weight
length = 4 #max is 7

R1 = [ ]
dim_R1 = 2**(length*w)
N1 = 1/np.sqrt(dim_R1)

for i in range(dim_R1):
    R1.append(Qobj(oracle(i+1,dim_R1)))

R2 = [ ]
dim_R2 = 2**(length*w)
N2 = 1/np.sqrt(dim_R2)

for i in range(dim_R2):
    R2.append(Qobj(oracle(i+1,dim_R2)))
    
index_range = range(2**(length*w))
target_position = []
for k in index_range:
    target_position.append(k*dim_R2 + k)

In [70]:
inversor = [ ]
for i in range(dim_R1*dim_R2):
    inversor.append([ ])
    for j in range(dim_R1*dim_R2):
        if i==j:
            if j in target_position:     #do the inversion on the well-constructed states in the direct product state
                inversor[i].append(-1)
            else:
                inversor[i].append(1)
        else:
            inversor[i].append(0)
inversor = Qobj(inversor)

R12 = 0
for i in range(dim_R1):
    for j in range(dim_R2):
        R12 = R12 + tensor(R1[i],R2[j])
R12 = N1*N2*R12
diffusor = Qobj(diffusor_construct(R12))
R12.dims = [[R12.dims[0][0]*R12.dims[0][1]],[1]]

In [71]:
# main iteration
p1 = []
p2 = []
p3 = []

num_iteration = int((dim_R1+dim_R2)**(1/3))
psi_0 = R12
for dummy in range(num_iteration):
    psi_0 = diffusor * inversor * psi_0
    # state_s = diffusor * inversor * state_s
p1.append(np.real(psi_0[0][0][0]* np.conj(psi_0[0][0][0])))
p2.append(np.real(psi_0[1][0][0]* np.conj(psi_0[1][0][0])))
p3.append(np.real(psi_0[17][0][0]* np.conj(psi_0[17][0][0])))

In [112]:
#Grover inversor
inversor_grover = [ ]
for i in range(dim_R2):
    inversor_grover.append([ ])
    for j in range(dim_R2):
        if i==j:
            if j == 0:
                inversor_grover[i].append(-1)
            else:
                inversor_grover[i].append(1)
        else:
            inversor_grover[i].append(0)
inversor_grover = tensor(qeye(dim_R1),Qobj(inversor_grover))
inversor_grover.dims = [[inversor_grover.dims[0][0]*inversor_grover.dims[0][1]],[inversor_grover.dims[0][0]*inversor_grover.dims[0][1]]]

#Grover diffusor
# state_s = 0
# for i in R2:
#     state_s += i
# state_s *= N2

state_s = psi_0
# diffusor_grover = tensor(qeye(dim_R1),Qobj(diffusor_construct(state_s)))
diffusor_grover = Qobj(diffusor_construct(state_s))
# diffusor_grover.dims = [[diffusor_grover.dims[0][0]*diffusor_grover.dims[0][1]],[diffusor_grover.dims[0][0]*diffusor_grover.dims[0][1]]]

In [113]:
psi_f = psi_0
num_iteration_grover = int((dim_R2)**(1/2))
p1_g = []
p2_g = []
p3_g = []
psi_record = []
for dummy in range(10):
    psi_f = psi_0
    for i in range(dummy):
        psi_f = diffusor_grover * inversor_grover * psi_f
    p1_g.append(np.real(psi_f[0][0][0]* np.conj(psi_f[0][0][0])))
    p2_g.append(np.real(psi_f[1][0][0]* np.conj(psi_f[1][0][0])))
    p3_g.append(np.real(psi_f[17][0][0]* np.conj(psi_f[17][0][0])))
    psi_record.append(psi_f)

In [139]:
for i in range(1,len(str(p2_g[5]))-4,1):
    print(str(p2_g[5])[i])

0
.
0
0
0
1
5
0
3
4
1
7
0
1
5
5
5
5
8
9
6
8


In [141]:
np.real(p2_g[0])

0.00016117095947265625

In [135]:
p2_g,p2

([(0.00016117095947265625+0j),
  (9.065866470336914e-05+0j),
  (1.5739351511001587e-05+0j),
  (6.6498760133981705e-06+0j),
  (7.19116214895621e-05+0j),
  (0.00015034170155558968+0j),
  (0.00016841191614958007+0j),
  (0.00010918143908966726+0j),
  (2.817884261951953e-05+0j),
  (1.3440609299003592e-06+0j)],
 [(0.00016117095947265625+0j)])

In [116]:
p3_g,p3

([(0.060082435607910156+0j),
  (0.03379637002944946+0j),
  (0.005867425352334976+0j),
  (0.0024789872113615274+0j),
  (0.026807716363691725+0j),
  (0.05604542922901601+0j),
  (0.06278176999609286+0j),
  (0.040701419195787736+0j),
  (0.010504705703386685+0j),
  (0.0005010484180156954+0j)],
 [(0.060082435607910156+0j)])

In [117]:
print(psi_record[0])

Quantum object: dims = [[256], [1]], shape = (256, 1), type = ket
Qobj data =
[[ 0.24511719]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [ 0.24511719]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [ 0.24511719]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [ 0.24511719]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01269531]
 [-0.01

In [118]:
print(psi_record[1])

Quantum object: dims = [[256], [1]], shape = (256, 1), type = ket
Qobj data =
[[ 0.67407227]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.03491211]
 [ 0.18383789]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.03491211]
 [-0.00952148]
 [ 0.18383789]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.03491211]
 [-0.00952148]
 [-0.00952148]
 [ 0.18383789]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00952148]
 [-0.00

In [119]:
print(psi_record[2])

Quantum object: dims = [[256], [1]], shape = (256, 1), type = ket
Qobj data =
[[ 0.93450928]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.04840088]
 [ 0.07659912]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.04840088]
 [-0.00396729]
 [ 0.07659912]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.04840088]
 [-0.00396729]
 [-0.00396729]
 [ 0.07659912]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00396729]
 [-0.00

In [120]:
print(psi_record[3])

Quantum object: dims = [[256], [1]], shape = (256, 1), type = ket
Qobj data =
[[ 0.96131897]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [-0.04978943]
 [-0.04978943]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [-0.04978943]
 [ 0.00257874]
 [-0.04978943]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [-0.04978943]
 [ 0.00257874]
 [ 0.00257874]
 [-0.04978943]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00257874]
 [ 0.00

In [121]:
print(psi_record[4])

Quantum object: dims = [[256], [1]], shape = (256, 1), type = ket
Qobj data =
[[ 0.74779892]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [-0.03873062]
 [-0.16373062]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [-0.03873062]
 [ 0.00848007]
 [-0.16373062]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [-0.03873062]
 [ 0.00848007]
 [ 0.00848007]
 [-0.16373062]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00848007]
 [ 0.00

In [122]:
print(psi_record[5])

Quantum object: dims = [[256], [1]], shape = (256, 1), type = ket
Qobj data =
[[ 0.34732914]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [-0.01798916]
 [-0.23673916]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [-0.01798916]
 [ 0.01226139]
 [-0.23673916]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [-0.01798916]
 [ 0.01226139]
 [ 0.01226139]
 [-0.23673916]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01226139]
 [ 0.01

In [87]:
print(psi_record[6])

Quantum object: dims = [[256], [1]], shape = (256, 1), type = ket
Qobj data =
[[-0.24902081]
 [ 0.0057075 ]
 [ 0.0057075 ]
 [ 0.0057075 ]
 [ 0.0057075 ]
 [ 0.0057075 ]
 [ 0.0057075 ]
 [ 0.0057075 ]
 [ 0.0057075 ]
 [ 0.0057075 ]
 [ 0.0057075 ]
 [ 0.0057075 ]
 [ 0.0057075 ]
 [ 0.0057075 ]
 [ 0.0057075 ]
 [ 0.0057075 ]
 [ 0.01451945]
 [ 0.23651767]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [ 0.01451945]
 [-0.02129483]
 [ 0.23651767]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [ 0.01451945]
 [-0.02129483]
 [-0.02129483]
 [ 0.23651767]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02129483]
 [-0.02

In [88]:
print(psi_record[7])

Quantum object: dims = [[256], [1]], shape = (256, 1), type = ket
Qobj data =
[[-0.20719165]
 [ 0.03612167]
 [ 0.03612167]
 [ 0.03612167]
 [ 0.03612167]
 [ 0.03612167]
 [ 0.03612167]
 [ 0.03612167]
 [ 0.03612167]
 [ 0.03612167]
 [ 0.03612167]
 [ 0.03612167]
 [ 0.03612167]
 [ 0.03612167]
 [ 0.03612167]
 [ 0.03612167]
 [ 0.00500327]
 [-0.24603385]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.00500327]
 [ 0.01177865]
 [-0.24603385]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.00500327]
 [ 0.01177865]
 [ 0.01177865]
 [-0.24603385]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01177865]
 [ 0.01

In [89]:
print(psi_record[8])

Quantum object: dims = [[256], [1]], shape = (256, 1), type = ket
Qobj data =
[[-0.11356457]
 [ 0.05750541]
 [ 0.05750541]
 [ 0.05750541]
 [ 0.05750541]
 [ 0.05750541]
 [ 0.05750541]
 [ 0.05750541]
 [ 0.05750541]
 [ 0.05750541]
 [ 0.05750541]
 [ 0.05750541]
 [ 0.05750541]
 [ 0.05750541]
 [ 0.05750541]
 [ 0.05750541]
 [-0.00576372]
 [ 0.23526685]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.00576372]
 [-0.02254565]
 [ 0.23526685]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.00576372]
 [-0.02254565]
 [-0.02254565]
 [ 0.23526685]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02254565]
 [-0.02

In [90]:
print(psi_record[9])

Quantum object: dims = [[256], [1]], shape = (256, 1), type = ket
Qobj data =
[[ 0.00845366]
 [ 0.06451281]
 [ 0.06451281]
 [ 0.06451281]
 [ 0.06451281]
 [ 0.06451281]
 [ 0.06451281]
 [ 0.06451281]
 [ 0.06451281]
 [ 0.06451281]
 [ 0.06451281]
 [ 0.06451281]
 [ 0.06451281]
 [ 0.06451281]
 [ 0.06451281]
 [ 0.06451281]
 [-0.01508979]
 [-0.24459292]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [-0.01508979]
 [ 0.01321958]
 [-0.24459292]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [-0.01508979]
 [ 0.01321958]
 [ 0.01321958]
 [-0.24459292]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01321958]
 [ 0.01